In [2]:
!pip install numpy==1.16.6
!pip install river

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 24.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for numpy: filename=numpy-1.16.6-cp39-cp39-linux_x86_64.whl size=3408386 sha256=d3686917c10d07d3a9a78a8ba392d6e10d9ff6fed8efdafe6cd9908219cba270
  Stored in directory: /tmp/pip-ephem-wheel-cache-4vh3xuw_/wheels/ce/72/38/377a1055ac9b575ebba22e507856aecdc37f9bf6372703e5f9
Successfully built numpy
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.3
    Uninstalling numpy-1.22.3:
      Successfully uninstalled numpy-1.22.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tifffile 2022.3.16 requires numpy>=1.19.2, but you have numpy 1.16.6 which is incompatible.
tables 3.7.0 requires numpy>=1.19.0, but you have numpy 1.16.6 which is incompatible.
statsmodels 0.13.2 requires numpy>=1.17, but you have n

In [3]:
#encoding={'Benign':0, 'FTP-BruteForce':1,'DoS attacks-GoldenEye':2, 'DoS attacks-Slowloris':3,'DoS attacks-SlowHTTPTest':4, 'DoS attacks-Hulk':5,"DDOS attack-HOIC":6,"Bot":7,"DDOS attack-LOIC-UDP":8}

In [1]:
from river import stream
from river import preprocessing
from river import compose
from river import metrics
from river import stats
from river import evaluate
import pandas as pd
from river.feature_selection import PoissonInclusion
from river.imblearn import HardSamplingClassifier
from river.linear_model import LogisticRegression
from river.optim import SGD
from imblearn.metrics import geometric_mean_score

In [2]:
data = pd.read_csv("preprocessed_multiclass_stream.csv")

In [3]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2)

In [4]:
stream_data = stream.iter_pandas(train.drop("Label", axis = 1), y = train["Label"])

In [5]:
test_data = stream.iter_pandas(test.drop("Label", axis = 1), y = test["Label"])

In [6]:
def construct_pipeline(model):
    return compose.Pipeline(
                PoissonInclusion(p=0.6), # feature selection to reduce training time
                HardSamplingClassifier( # imblearn wrapper to solve class imbalance
                            classifier=model, 
                            p=0.1,
                            size=40,
                        )
                )

In [7]:
SGD_model = LogisticRegression(SGD(0.2))

In [8]:
model=construct_pipeline(SGD_model)

In [9]:
for x, y in stream_data:
    model = model.learn_one(x, y)      # make the model learn  

In [10]:
Acc_metric = metrics.Accuracy();
Rec_metric = metrics.Recall();
Pre_metric = metrics.Precision();
F1_metric = metrics.F1();

def get_metric(Acc, rec, pre, f1 ):
  Acc, rec, pre, f1 = Acc, rec, pre, f1
  return([Acc, rec, pre, f1])

predictions = []
true_y = []
for x, y in test_data:
    y_pred = model.predict_one(x) # make a prediction
    predictions.append(y_pred)
    true_y.append(y)
    Acc_metric = Acc_metric.update(y, y_pred)
    Rec_metric = Rec_metric.update(y, y_pred)
    Pre_metric = Pre_metric.update(y, y_pred)
    F1_metric = F1_metric.update(y, y_pred)
    model = model.learn_one(x, y)
print(get_metric(Acc_metric, Rec_metric, Pre_metric, F1_metric))  

[Accuracy: 17.51%, Recall: 99.77%, Precision: 9.72%, F1: 17.72%]


In [11]:
from river.metrics import ClassificationReport

report = ClassificationReport()
for yt,yp in zip(true_y, predictions):
  report = report.update(yt,yp)
print(report)
print("geometric mean score :",geometric_mean_score(true_y, predictions, average='weighted'))

           Precision   Recall   F1       Support  
                                                  
       0      65.24%   70.46%   67.75%       811  
       1       9.72%   99.77%   17.72%       523  
       2       0.00%    0.00%    0.00%      8232  
       3       0.00%    0.00%    0.00%      2108  
       4       0.00%    0.00%    0.00%       395  
       5       0.00%    0.00%    0.00%      3112  
       6       0.00%    0.00%    0.00%      1701  
       7       0.00%    0.00%    0.00%     14298  
       8       0.00%    0.00%    0.00%       373  
                                                  
   Macro       8.33%   18.91%    9.50%            
   Micro      17.51%   17.51%   17.51%            
Weighted       9.29%   17.51%   10.29%            

                 17.51% accuracy                  
geometric mean score : 0.40177553394397236
